# Importing Libraries

In [49]:
import tensorflow as tf
import json
import matplotlib.pyplot as plt
import pydot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Conv3D, Reshape, Activation, Dropout, Flatten
from tensorflow.config.experimental import set_memory_growth, get_visible_devices
from tensorflow.keras.utils import plot_model

# GPU Configuration
(You can skip this part. It will not cause issues even if you do not have a CUDA enabled GPU.)

In [4]:
gpus=get_visible_devices('GPU')
for gpu in gpus:
    set_memory_growth(gpu, True)

# Defining general model variables

In [36]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = False

# Dataset

## Import

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 160s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Preprocessing

In [10]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [34]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Model definition

## Normal Model

In [20]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [22]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
plot_model(model, "resources/2d.png", show_shapes=True)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_8 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)       

## New Approach

In [44]:
model1 = Sequential()
model1.add(Conv3D(32, 3,
                 input_shape=(32,32,3,1)))
model1.add(Reshape((30,30,32)))
model1.add(Activation('relu'))
model1.add(Conv2D(32, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(64, (3, 3), padding='same'))
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3)))
model1.add(Activation('relu'))
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(512))
model1.add(Activation('relu'))
model1.add(Dropout(0.5))
model1.add(Dense(num_classes))
model1.add(Activation('softmax'))

In [45]:
opt1 = tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

In [46]:
model1.compile(loss='categorical_crossentropy',
              optimizer=opt1,
              metrics=['accuracy'])

plot_model(model1, "resources/3d.png", show_shapes=True)
model1.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 30, 30, 1, 32)     896       
_________________________________________________________________
reshape_2 (Reshape)          (None, 30, 30, 32)        0         
_________________________________________________________________
activation_24 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_25 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 32)      

# Training

## General model

In [39]:
x_train=x_train.reshape((-1,32,32,3))
x_test=x_test.reshape((-1,32,32,3))
if not data_augmentation:
    print('Not using data augmentation.')
    h=model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
    datagen.fit(x_train)
    h=model.fit(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),)
h=h.history

Not using data augmentation.
Epoch 1/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8111 - accuracy: 0.3354 - val_loss: 1.5097 - val_accuracy: 0.4536
Epoch 2/100
1563/1563 [==============================] - 7s 5ms/step - loss: 1.4839 - accuracy: 0.4650 - val_loss: 1.3055 - val_accuracy: 0.5348
Epoch 3/100
1563/1563 [==============================] - 7s 5ms/step - loss: 1.3330 - accuracy: 0.5260 - val_loss: 1.3078 - val_accuracy: 0.5254
Epoch 4/100
1563/1563 [==============================] - 7s 5ms/step - loss: 1.2235 - accuracy: 0.5692 - val_loss: 1.1157 - val_accuracy: 0.6092
Epoch 5/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1500 - accuracy: 0.5964 - val_loss: 1.0401 - val_accuracy: 0.6381
Epoch 6/100
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0821 - accuracy: 0.6230 - val_loss: 1.0571 - val_accuracy: 0.6332
Epoch 7/100
1563/1563 [==============================] - 7s 4ms/step - loss: 1.0337 - accuracy: 0.638

## Modified model

In [47]:
x_train=x_train.reshape((-1,32,32,3,1))
x_test=x_test.reshape((-1,32,32,3,1))
if not data_augmentation:
    print('Not using data augmentation.')
    h1=model1.fit(x_train1, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test1, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)
    datagen.fit(x_train)
    h1=model1.fit(datagen.flow(x_train.reshape(-1,32,32,3,1), y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),)
h1=h1.history

Not using data augmentation.
Epoch 1/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8285 - accuracy: 0.3276 - val_loss: 1.5933 - val_accuracy: 0.4246
Epoch 2/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.5392 - accuracy: 0.4409 - val_loss: 1.3865 - val_accuracy: 0.4979
Epoch 3/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.4065 - accuracy: 0.4951 - val_loss: 1.2805 - val_accuracy: 0.5414
Epoch 4/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2995 - accuracy: 0.5367 - val_loss: 1.1805 - val_accuracy: 0.5793
Epoch 5/100
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2165 - accuracy: 0.5687 - val_loss: 1.1221 - val_accuracy: 0.6054
Epoch 6/100
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1527 - accuracy: 0.5922 - val_loss: 1.0517 - val_accuracy: 0.6314
Epoch 7/100
1563/1563 [==============================] - 9s 5ms/step - loss: 1.0958 - accuracy: 0.615

# Model Metrics

In [ ]:
fig=plt.figure()
tra=fig.add_subplot(121)
tra.plot(h["accuracy"], label="2D acc", color="#B0E0E6")
tra.plot(h["val_accuracy"], label="2D val_acc", color="#FFDB99")
tra.plot(h1["accuracy"], label="3D acc", color="#379FAE")
tra.plot(h1["val_accuracy"], label="3D val_acc", color="#FFA500")
tra.legend()
tra1=fig.add_subplot(121)
tra1.plot(h["loss"], label="2D acc", color="#B0E0E6")
tra1.plot(h["val_loss"], label="2D val_acc", color="#FFDB99")
tra1.plot(h1["loss"], label="3D acc", color="#379FAE")
tra1.plot(h1["val_loss"], label="3D val_acc", color="#FFA500")
tra1.legend()
fig.suptitle("Model metrics")
plt.savefig("resources/metrics.png",dpi=300)
plt.show()

# Saving model metrics

In [ ]:
f=open("resources/2d.json","w")
json.dump(h,f)
f.close()

In [ ]:
f=open("resources/3d.json","w")
json.dump(h1,f)
f.close()

# Saving Models

In [ ]:
model.save("models/c2d.h5")

In [ ]:
model1.save("models/c3d.h5")

# Loading model metrics

In [ ]:
f=open("resources/2d.json","r")
h=json.load(f)
f.close()

In [ ]:
f=open("resources/3d.json","r")
h1=json.load(f)
f.close()

# Scoring the models

In [ ]:
scores = model.evaluate(x_test.reshape((-1,32,32,3)), y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
scores = model1.evaluate(x_test.reshape((-1,32,32,3,1)), y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])